# EchoWare

Making robots more productive has been the purpose of the feild for as long as it has existed. The question how productive is a robot is too vague so the research tends to focus on how robots can accomplish tasks. Via mastering tasks, systems of robots can be implemented to provide higher levels of productivity than any group of humans could. this is shown by the heavy use of automation in many industries: manufacturing, analytics, monitoring/reporting and even landscaping has seen productivity boosts from incorporating agricultural robots (like Verdant and Scythe). Teaching robots to perform tasks is well researched and mostly solved problem. The current direction of research is implementing a way to learn many tasks without any prior knowledge of the tasks. This allows robots to improve their productivity with minimal intervention, causing operators to spend less time training robots and more time being productive.

There are two current trends for implementing learning capabilities described above. The first being Imitation Learning and the second is to use a DNN too learn end-to-end behavior replication. Using an end-to-end NN has become a popular default for robotics problems that require advanced mathematical intuition. The DNN requires the user needs to crete a dataset, which requires knowing the dynamics. This method is less promising because the user needs to build a valid dataset that encapsulates the desired behavior (a chicken and egg paradox). Imitation Learning instead of finding a maping between datasets will learn a reward function from expert data and use it to train a policy. Imitation Learning is more promising because it does not require any prior information about the agent, the system will learn behaviors from the expert.

There has been lots of work done on methods for learning the reward of an expert. The more advanced methods have shown that the trained policy can improve on the behavior of the expert to a certain degree. This is because the agent is not learning directly from the experts action. Instead the agent is learning from the reward function that the expert seems to be using. With more expert data the estimation of the reward function gets better. The biggest example of where the policy breaks down was a self driving car. The car was trained on expert driver data, the experts never crashed and thus never needed to recover. The agent learned how to drive very well but could not recover. In order to learn robust behavior, the dataset must posses a complete distribution of scenarios.

With the powerful intuition of IL, training robots to perform tasks has become scalable, robust and "lower" effort. All that is required for this system to learn a new task is to create a dataset of experts performing said task. The Echoware team sees this as a data generation problem. There are many methods availble for capturing realtime poses and converting them to an "expert dataset" (MoCap being the best for obvious reasons). With plenty of expert data available it seems nothing should be holding the feild back from robots that can learn anything. However, one of the constraints for an agent to properly learn a behavior is the shape of the expert needs to be close to the agent. This is partly because most implementation are learning a policy that maps perception and localization to actuator outputs.

The Echoware team is proposing a means of data generation to solve this problem. The original goal is a generic method for retargting an expert's poses to an agent's pose. This would provide a systematic way of generating usuable agent data from a dissimilar expert. This approach has problems with being able to create a physically useful representation without a deep understanding of both the expert and agent. A second proposal is to remove the part that makes the agent and expert dissimilar. If the IL is used to train a much higher level policy, the differences between expert and agent will not matter as much. The challenges here being a fairly new direction for the feild and the dependance on a high level task aware abstraction language, there are examples of these which build graphs of contact points and robots move by traversing this graph. This is similar to how Cerberus handled the SUB-T DARPA chanllenge.

The expert and agent poses consist of a configuration space, set of links and a set of joint transforms.
$
    \{\mathbb{C}_e,\mathbb{L}_e,\mathbb{J}_e\} \rightarrow \{\mathbb{C}_a,\mathbb{L}_a,\mathbb{J}_a\}
$

Where

$ \mathbb{C} $: Configuration, DOF x 1 vector 

$ \mathbb{L} $: Link set, {(Polygon3D, &Joint, ...), ...} 

$ \mathbb{J} $: Joint set, {(Translation3D, state, Axis3D, effect, &Link), ...}


> **Configuration** of any robot can and should be implemented as a list of actuator states. This vector is the input to forward kinematics and the output of inverse kinematics.  

> **Links** provide a geometric definition for the parts of a robot. A link consists of a polygon (convex hull of 3D points) and a possibly empty set of joint references.

> **Joints** describe a state, axis and motion of the joint on the effected link. Most joints will be continuous revolute about their Z axis (standard way to define motor reference frames). Each joint refers to a single link, creating chains: link -> joint -> link -> joint -> link. The axis of the joint is a normal vector oriented along the axis of rotation (revolute) or linear motion (linear).

For this project we will assume a set of rules the expert and agent must follow:
- A Link can have any number of Joints
- No set of links/joints forms a cycle
- All chains end in a Link/Every joint has a base link and an effected link
- A proper mapping should be independant of configuration


## Example Definition
![sample robot arm](two_link_basic.png)
> 2 DOF, 3 link robot arm (joints at $P_1$ and $P_2$, ee at $P_3$)

The above robot has the configuration space $$ \begin{bmatrix}\theta_1 \\ \theta_2 \end{bmatrix} $$

The linkages can be defined as line segments connecting the two joints and end effector: $$ \{ Link_0: ((P_0, P_1), Joint_1), Link_1: ((P_1, P_2), Joint_2),  Link_2: ((P_2, P_3), NULL)\} $$
*the points in this definition are defined in the link's reference frame. This means the first point in the segment is always at the origin and the second point is the length of the link along the X axis. The true orientation and position of the links is found using kinematics and requires knowledge of the joint states*

The Joints are offset from the links by the line segment that defines the links. This allows a very simple Transform that translates by the vector difference of the end points (or the length of the link along the X axis). $$ \{Joint_1: (P_1-P_0, \theta_1, [0,0,1], revolute, Link_1), Joint_2: (P_2-P_1, \theta_2, [0,0,1], revolute, Link_2)\} $$

### Forward Kinematics
In order to solve the forward kinematics for the end effector, start with a vector from $Link_n$ to the end effector ($P_3-P_2$ in this case). Apply the effect of $Joint_n$ to this vector (rotation about Z) and add the transform to $Link_{n-1}$ ($P_2-P_1$), this is the end effector defined relative to $Link_{n-1}$. Repeat this process untill the end effector is defined about $Link_0$.

## Retargeting Goals
To retarget this pose we should consider what the goal of retargeting is. Ideally the agent can reproduce results of an expert's actions. Due to the complexity of considering the environment, this project simplifies the goal to imitating an expert's pose. Work done by students at CMU showed that imitating an experts pose well enough can lead to an agent accomplishing tasks. Future work should require an agent also diverge from the expert enough to be in a feasible pose. This addition would look like physical constraints for the retargeting (no self collisions, match COM/momentum/forces).

## Retargeting Delivarables
An efficient and easy to implement mapping that can represent the result of this algorithm is the matrix $ \mathbb{M}_e^a $. Which is implemented as $ \mathbb{C}_a = \mathbb{M}_e^a * \mathbb{C}_e $. This claims any joint on the agent is a linear combiniation of joints on the expert. Also note $ \mathbb{M}_e^a \in \mathbb{R}^{n,m}$, where n is the agent DOF and m is the expert DOF (ie a cell relating each expert and agent joint). The algorithm below will descibe how to asssign weights to the cells of $ \mathbb{M}_e^a $.


## Joint/Link Matching
*Joints with the same depth/transform will match* 

    // does not require kinematics or optimization
    // uses thresholds and distances for similarities
    // subject to biasing
    match_joints(link_e, link_a, M):
        
        for joint_a in link_a:
        
            for joint_e in link_e:

                // joint is defined with a similar orientation and translation
                // always between 0 and 1, but thrown out if < joint threshold
                let weight = joint_e ~= joint_a
                
                if weight > joint_threshold

                    M(joint_e, joint_a) = weight

                    match_joints(joint_e.link, joint_a.link, M) // continue along current chain

                else:
                    
                    // match_joints(link_e, joint_a.link, M) // try adjacent links
                    match_joints(joint_e.link, link_a, M) // try adjacent links
                    
    M = zeros(n,m) // default weights of zero for all joint relationships
    match_joints(expert_base, agent_base, M)
                

    
Beginning from the base link of each pose, compare all joints and assign a weight to each comparison. Any joints whose match weight is greater than a threshold will then check each effected link. If a joint is not a good match it's effected link is compared with the expert's base link and the agents base link is compared with the expert's effected link. This allows joints to match at different depths on a chain.

> This version of retargeting expects very similar experts and agents
> The weighting of cells is not regulated from a high level and end up in too much weight for a joint
> Knowing the threshold it is easy to predict which joints will match


### Shape Matching
*Realtime retarget, expert pose -> Geometry => agent kinematics + Solver => agent configuration*
*O(N)*
Match poses by matching geometries. Align the agent's joints/ee to be on the surface of $ConvexHull(expert_joints)$. Start by aligning the agent and expert's base links. Adjust the agent's joints to minimize the error in link projected com (*com projected onto the joint->joint line, keeps the link out of the middle of the geometry*) and immediate child joints to the surface (error must be orthogonal to joint axis).

### End Effector/Chain matching
*Chains will match end effector positions* \
Begin with the expert's base link and match any joints (ignoring links). Then select from the following for each joint:
1. Use IK of agent to create similar ee position or shape of multiple ee (likely doesn't use the same shape)
2. Use optimization to minimize the error in keypoints on the chain (similar to shape matching)
3. Use the closing circle (sphere) algorithm (put all joint positions and desired ee on a circle and shrink the circle until they meet)

> Both require normalizing each chain definition so numerically chains have the same reach

In [1]:
# This allows file changes to be included without reloading the kernel
# Still requires reimporting the modules

%load_ext autoreload 
%autoreload 2   

In [34]:
import numpy as np
from pose import Transform3D

In [32]:
tf = Transform3D(1,0,0,1.57,-3.1415,1.57)

In [33]:
v = np.array([1 ,0 ,0])
tf.transform(v)

array([2.66453526e-15, 3.41810697e-12, 7.37825283e-08])